# Modeling

In [1]:
import sys

sys.path.append('/Users/nickolaspedrimiranda/CodeUp/Regression-Project/')

In [2]:
# Import basic DS libraries
import pandas as pd
import numpy as np

# Import libraries for visuals
import matplotlib.pyplot as plt
import seaborn as sns

# Import useful DS functions
import acquire as ac
import prepare as prep
import evaluate as ev
import explore as ex

# For modeling
from sklearn.linear_model import LinearRegression, LassoLars, TweedieRegressor
from sklearn.preprocessing import PolynomialFeatures

from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor

from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression

seed = 100

In [3]:
wine = ac.get_wine()

In [4]:
wine.head()

,fixed_acidity,volatile_acidity,citric_acid,residual_sugar,chlorides,free_sulfur_dioxide,total_sulfur_dioxide,density,pH,sulphates,alcohol,quality,wine_type
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5,red
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5,red
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5,red
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6,red
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5,red


In [5]:
wine = ex.reajust_range(wine)
# wine = wine.drop(columns='residual_sugar')

In [6]:
train, val, test = prep.train_val_test(wine, strat='quality', seed=seed)
train, val, test = prep.scale(wine, train=train, val=val, test=test, scaled_cols=(wine.select_dtypes(float).columns))
train, val, test = prep.dummies(train, val, test, drop_first=['wine_type'])
x_train, y_train = prep.split_xy(train, 'quality')
x_val, y_val = prep.split_xy(val, 'quality')

In [7]:
train.head()

,fixed_acidity,volatile_acidity,citric_acid,residual_sugar,chlorides,free_sulfur_dioxide,total_sulfur_dioxide,density,pH,sulphates,alcohol,quality,wine_type_white
324,0.739726,0.573427,0.253165,0.514851,0.563636,0.152866,0.173913,0.928986,0.400000,0.610390,0.137931,6,0
5697,0.273973,0.181818,0.430380,0.247525,0.336364,0.369427,0.529644,0.379600,0.457143,0.207792,0.448276,5,1
2405,0.246575,0.321678,0.329114,0.623762,0.336364,0.713376,0.786561,0.670755,0.409524,0.246753,0.206897,5,1
3107,0.301370,0.181818,0.620253,0.688119,0.463636,0.624204,0.648221,0.774048,0.771429,0.285714,0.241379,6,1
5166,0.315068,0.251748,0.341772,0.178218,0.200000,0.242038,0.292490,0.244028,0.361905,0.220779,0.534483,6,1


## BASELINE

In [8]:
results = ev.baseline(train.quality)

In [9]:
results.head()

,quality,base_median,base_mean
324,6,6.0,5.83048
5697,5,6.0,5.83048
2405,5,6.0,5.83048
3107,6,6.0,5.83048
5166,6,6.0,5.83048


In [10]:
SSE, MSE, RMSE = ev.eval_model(results.quality, results.base_mean.round())
SSE, MSE, RMSE

(3193.0, 0.7591535901093676, 0.8712942041063785)

In [11]:
SSE, MSE, RMSE = ev.eval_model(results.quality, results.base_median)
SSE, MSE, RMSE

(3193.0, 0.7591535901093676, 0.8712942041063785)

## POLYNOMIAL FEATURES

In [12]:
poly = PolynomialFeatures()
x_train_s = poly.fit_transform(x_train)
x_val_s = poly.transform(x_val)

## Linear Regression

In [13]:
lm = LinearRegression()  # Create model object
ev.train_model_gen2(lm, x_train, y_train, x_val, y_val)  # Check train and validate

The train RMSE is 0.7795532573385248.
The validate RMSE is 0.7684107299774686.


In [14]:
lm = LinearRegression()  # Create model object
ev.train_model_gen2(lm, x_train_s, y_train, x_val_s, y_val)  # Check train and validate

The train RMSE is 0.7426930472507584.
The validate RMSE is 0.7479160837744295.


## XGBRegressor

In [15]:
xgbr = XGBRegressor(max_depth=8, n_estimators=8, random_state=100)  # Create best model

ev.train_model_gen2(xgbr, x_train, y_train, x_val, y_val) # Evaluate

The train RMSE is 0.6699869765449888.
The validate RMSE is 0.7988615651174962.


In [16]:
xgbr = XGBRegressor(max_depth=8, n_estimators=8, random_state=100)  # Create best model

ev.train_model_gen2(xgbr, x_train_s, y_train, x_val_s, y_val) # Evaluate

The train RMSE is 0.6251961175897992.
The validate RMSE is 0.7995559255833236.


## Random Forest Regressor

In [17]:
# Create best model
rf = RandomForestRegressor(max_depth=6, min_samples_split=8, random_state=100)

ev.train_model_gen2(rf, x_train, y_train, x_val, y_val)  # Evaluate model

The train RMSE is 0.6810735281429181.
The validate RMSE is 0.7351942218490143.


In [18]:
# Create best model
rf = RandomForestRegressor(max_depth=6, min_samples_split=8, random_state=100)

ev.train_model_gen2(rf, x_train_s, y_train, x_val_s, y_val)  # Evaluate model

The train RMSE is 0.6715819749108831.
The validate RMSE is 0.732168718388364.


## LASSO LARS

In [19]:
ll = LassoLars(alpha=0)
ev.train_model(ll, x_train, y_train, x_val, y_val)

The train RMSE is 0.716771885110661.
The validate RMSE is 0.7163999296023469.


## KNN

In [20]:
knn = KNeighborsClassifier(n_neighbors=7, weights='uniform')
ev.train_model(knn, x_train, y_train, x_val, y_val)

The train RMSE is 0.7062656880124635.
The validate RMSE is 0.784137975129349.


In [21]:
knn = KNeighborsClassifier(n_neighbors=7, weights='uniform')
ev.train_model(knn, x_train_s, y_train, x_val_s, y_val)

The train RMSE is 0.7094565372322593.
The validate RMSE is 0.7791682864734351.


## Random Forest Classifier

In [22]:
rf1 = RandomForestClassifier(max_depth=5, random_state=seed)
ev.train_model(rf1, x_train, y_train, x_val, y_val)

The train RMSE is 0.7635291087787347.
The validate RMSE is 0.7684107299774686.


In [23]:
rf1 = RandomForestClassifier(max_depth=5, random_state=seed)
ev.train_model(rf1, x_train_s, y_train, x_val_s, y_val)

The train RMSE is 0.7328641563154681.
The validate RMSE is 0.7508781392624222.


## Decision Tree

In [24]:
clf = DecisionTreeClassifier(max_depth=3, random_state=seed)
ev.train_model(clf, x_train, y_train, x_val, y_val)

The train RMSE is 0.7977917692512981.
The validate RMSE is 0.8016353983424719.


## Logistic Regression

RESULTS WITH SUGAR: 

lm = 76, 74
xgb = 79, 79
random forest regressor = 73, 73
KNN = 78.5
rf1 = 76
decision tree = 80
logreg= 79

WITHOUT SUGAR

lm = 78, 76
xgb = 80, 83
random forest regressor = 73, 73
KNN = 78.5
rf1 = 76
decision tree = 80
logreg= 79

Without sugar is slightly worse

In [25]:
def test_models(x_train, y_train, x_val, y_val, d=6):
    poly = PolynomialFeatures()
    x_train_s = poly.fit_transform(x_train)
    x_val_s = poly.transform(x_val)
    
    print('Linear Regression')
    lm = LinearRegression()  # Create model object
    ev.train_model(lm, x_train, y_train, x_val, y_val)  # Check train and validate
    ev.train_model(lm, x_train_s, y_train, x_val_s, y_val)  # Check train and validate
    print()
    print('XGBRegressor')
    xgbr = XGBRegressor(max_depth=d, random_state=100)  # Create best model
    ev.train_model(xgbr, x_train, y_train, x_val, y_val) # Evaluate
    ev.train_model(xgbr, x_train_s, y_train, x_val_s, y_val) # Evaluate
    print()
    print('Random Forest Regressor')
    # Create best model
    rf = RandomForestRegressor(max_depth=6, min_samples_split=8, random_state=100)
    ev.train_model(rf, x_train, y_train, x_val, y_val)  # Evaluate model
    ev.train_model(rf, x_train_s, y_train, x_val_s, y_val)  # Evaluate model
    print()
    print('KNN')
    knn = KNeighborsClassifier(n_neighbors=7, weights='uniform')
    ev.train_model(knn, x_train, y_train, x_val, y_val)
    ev.train_model(knn, x_train_s, y_train, x_val_s, y_val)
    print()
    print('Random Forest Classifier')
    rf1 = RandomForestClassifier(max_depth=d, random_state=seed)
    ev.train_model(rf1, x_train, y_train, x_val, y_val)
    ev.train_model(rf1, x_train_s, y_train, x_val_s, y_val)
    print()
    print('Decision Tree Classifier')
    clf = DecisionTreeClassifier(max_depth=d, random_state=seed)
    ev.train_model(clf, x_train, y_train, x_val, y_val)
    ev.train_model(clf, x_train_s, y_train, x_val_s, y_val)
    print()
    print()
    

In [26]:
test_models(x_train, y_train, x_val, y_val)

Linear Regression
The train RMSE is 0.716771885110661.
The validate RMSE is 0.7163999296023467.
The train RMSE is 0.6742875122115677.
The validate RMSE is 0.6800919089836661.

XGBRegressor
The train RMSE is 0.24108011606029658.
The validate RMSE is 0.6387545348940477.
The train RMSE is 0.13036389272230306.
The validate RMSE is 0.621414587343548.

Random Forest Regressor
The train RMSE is 0.6190106752910569.
The validate RMSE is 0.6783116786928595.
The train RMSE is 0.6034036269943391.
The validate RMSE is 0.6715200733143334.

KNN
The train RMSE is 0.7062656880124635.
The validate RMSE is 0.784137975129349.
The train RMSE is 0.7094565372322593.
The validate RMSE is 0.7791682864734351.

Random Forest Classifier
The train RMSE is 0.7263467822753336.
The validate RMSE is 0.7560338123498209.
The train RMSE is 0.7020451070852293.
The validate RMSE is 0.7382073255487889.

Decision Tree Classifier
The train RMSE is 0.7407697991869955.
The validate RMSE is 0.7741666960364172.
The train RMSE is 

In [37]:
wine = prep.pour()

In [38]:
wine.head()

,fixed_acidity,volatile_acidity,citric_acid,residual_sugar,chlorides,free_sulfur_dioxide,total_sulfur_dioxide,density,pH,sulphates,alcohol,quality,wine_type
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5,red
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5,red
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6,red
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5,red
5,7.4,0.66,0.00,1.8,0.075,13.0,40.0,0.9978,3.51,0.56,9.4,5,red


In [39]:
wine = wine.drop(columns=['density'])

In [40]:
x_train, y_train, x_val, y_val, test = prep.drink_up(wine)

In [31]:
test_models(x_train, y_train, x_val, y_val)

Linear Regression
The train RMSE is 0.7197657355454814.
The validate RMSE is 0.7197657355454814.
The train RMSE is 0.6818510663041247.
The validate RMSE is 0.6818510663041247.

XGBRegressor
The train RMSE is 0.239043929286722.
The validate RMSE is 0.239043929286722.
The train RMSE is 0.14088309676849828.
The validate RMSE is 0.14088309676849828.

Random Forest Regressor
The train RMSE is 0.619751745996752.
The validate RMSE is 0.619751745996752.
The train RMSE is 0.6032898823238895.
The validate RMSE is 0.6032898823238895.

KNN
The train RMSE is 0.7044117479172953.
The validate RMSE is 0.7044117479172953.
The train RMSE is 0.7091213345028308.
The validate RMSE is 0.7091213345028308.

Random Forest Classifier
The train RMSE is 0.7286344943408314.
The validate RMSE is 0.7286344943408314.
The train RMSE is 0.6993305677476211.
The validate RMSE is 0.6993305677476211.

Decision Tree Classifier
The train RMSE is 0.7426930472507584.
The validate RMSE is 0.7426930472507584.
The train RMSE is 0

In [43]:
poly = PolynomialFeatures()
x_train_s = poly.fit_transform(x_train)
x_val_s = poly.transform(x_val)

xgbr = XGBRegressor(max_depth=6, random_state=100)  # Create best model
ev.train_model(xgbr, x_train, y_train, x_val, y_val) # Evaluate
ev.train_model(xgbr, x_train_s, y_train, x_val_s, y_val) # Evaluate

xgbr = XGBRegressor(max_depth=6, random_state=100)  # Create best model
ev.train_model_gen2(xgbr, x_train, y_train, x_val, y_val) # Evaluate
ev.train_model_gen2(xgbr, x_train_s, y_train, x_val_s, y_val) # Evaluate

The train RMSE is 0.239043929286722.
The validate RMSE is 0.239043929286722.
The train RMSE is 0.14088309676849828.
The validate RMSE is 0.14088309676849828.
The train RMSE is 0.23787776223724555.
The validate RMSE is 0.23787776223724555.
The train RMSE is 0.08585116892008703.
The validate RMSE is 0.08585116892008703.


In [44]:
xgbr = XGBRegressor(max_depth=6, random_state=100)  # Create best model
xgbr.fit(x_train, y_train)
train_prediction = xgbr.predict(x_train)
val_prediction = xgbr.predict(x_val)

In [47]:
y_val

324     6
5697    5
2405    5
3107    6
5166    6
       ..
458     7
100     6
4338    7
6439    7
6113    5
Name: quality, Length: 4206, dtype: int64

In [52]:
val_results = pd.DataFrame(y_val)

In [53]:
val_results['val_prediction'] = val_prediction

In [54]:
val_results

,quality,val_prediction
324,6,5.953650
5697,5,5.648510
2405,5,5.406800
3107,6,5.947033
5166,6,6.111743
...,...,...
458,7,6.875203
100,6,5.610467
4338,7,7.017803
6439,7,6.893229


In [55]:
x_train

,fixed_acidity,volatile_acidity,citric_acid,residual_sugar,chlorides,free_sulfur_dioxide,total_sulfur_dioxide,pH,sulphates,alcohol,wine_type_white
324,0.739726,0.573427,0.253165,0.514851,0.563636,0.152866,0.173913,0.400000,0.610390,0.137931,0
5697,0.273973,0.181818,0.430380,0.247525,0.336364,0.369427,0.529644,0.457143,0.207792,0.448276,1
2405,0.246575,0.321678,0.329114,0.623762,0.336364,0.713376,0.786561,0.409524,0.246753,0.206897,1
3107,0.301370,0.181818,0.620253,0.688119,0.463636,0.624204,0.648221,0.771429,0.285714,0.241379,1
5166,0.315068,0.251748,0.341772,0.178218,0.200000,0.242038,0.292490,0.361905,0.220779,0.534483,1
...,...,...,...,...,...,...,...,...,...,...,...
458,0.794521,0.349650,0.797468,0.108911,0.681818,0.050955,0.063241,0.495238,0.675325,0.482759,0
100,0.506849,0.741259,0.379747,0.074257,0.681818,0.127389,0.173913,0.628571,0.506494,0.310345,0
4338,0.301370,0.335664,0.354430,0.207921,0.227273,0.305732,0.371542,0.323810,0.324675,0.689655,1
6439,0.301370,0.363636,0.455696,0.410891,0.181818,0.547771,0.482213,0.514286,0.168831,0.612069,1


In [56]:
x_val

,fixed_acidity,volatile_acidity,citric_acid,residual_sugar,chlorides,free_sulfur_dioxide,total_sulfur_dioxide,pH,sulphates,alcohol,wine_type_white
324,0.739726,0.573427,0.253165,0.514851,0.563636,0.152866,0.173913,0.400000,0.610390,0.137931,0
5697,0.273973,0.181818,0.430380,0.247525,0.336364,0.369427,0.529644,0.457143,0.207792,0.448276,1
2405,0.246575,0.321678,0.329114,0.623762,0.336364,0.713376,0.786561,0.409524,0.246753,0.206897,1
3107,0.301370,0.181818,0.620253,0.688119,0.463636,0.624204,0.648221,0.771429,0.285714,0.241379,1
5166,0.315068,0.251748,0.341772,0.178218,0.200000,0.242038,0.292490,0.361905,0.220779,0.534483,1
...,...,...,...,...,...,...,...,...,...,...,...
458,0.794521,0.349650,0.797468,0.108911,0.681818,0.050955,0.063241,0.495238,0.675325,0.482759,0
100,0.506849,0.741259,0.379747,0.074257,0.681818,0.127389,0.173913,0.628571,0.506494,0.310345,0
4338,0.301370,0.335664,0.354430,0.207921,0.227273,0.305732,0.371542,0.323810,0.324675,0.689655,1
6439,0.301370,0.363636,0.455696,0.410891,0.181818,0.547771,0.482213,0.514286,0.168831,0.612069,1
